## House Prices Kaggle Competition
_______

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

In [2]:
# load data
train_df = pd.read_csv('data/train.csv', index_col='Id')
test_df = pd.read_csv('data/test.csv', index_col='Id')
X_test = test_df.copy()

In [3]:
train_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
all(np.array(test_df.columns)) == all(np.array(train_df.columns[:-1]))

True

In [5]:
# select target
y = train_df.SalePrice

#### I will use XGBoost to make predictions for house prices.
XGBoost can't deal with missing values and categorical varibales, so I need to preprocess data.

Firstly, I need to split train data into 'categorical' and 'numerical' variables, then find variables with missing values. Also I will divide categorical variable into 2 groups by the way I will encode them - ordinaly or with one-hot encoding.

Firstly, define the lists:
- `cols_num` - numerical variables
- `cols_cat_ord` - categorical variables for ordinal encoding
- `cols_cat_oh` - categorical variables for One-Hot encoding `cols_oh`

In [6]:
cols_cat_ord = []
cols_cat_oh = []
cols_num = []

#### Finding columns with missing Data

In [7]:
X_full = train_df.drop('SalePrice', axis=1)

In [8]:
cols_with_missing_values = [col for col in X_full.columns if X_full[col].isna().any()]

In [9]:
# we have several columns with missing values
cols_with_missing_values

['LotFrontage',
 'Alley',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [10]:
# let's find out how many missing values are there in each column
X_full[cols_with_missing_values].isnull().sum().sort_values(ascending=False)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
BsmtExposure      38
BsmtFinType2      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
MasVnrType         8
Electrical         1
dtype: int64

In [11]:
X_full.shape

(1460, 79)

#### Let's go through each column with missing values and decide what to do

__1) 'PoolQC'__

This is a categorical variable. According to description 'PoolQC' is a column with pool quality and NAs mean that there's no pool in the house.
But there are only few real values the rest are missing, so I will just drop this column

In [12]:
X_full.PoolQC.value_counts(dropna=False)

NaN    1453
Gd        3
Ex        2
Fa        2
Name: PoolQC, dtype: int64

In [13]:
X_full.PoolQC.isna().sum() / X_full.shape[0]

0.9952054794520548

In [14]:
X_full = X_full.drop('PoolQC', axis=1)
X_test = X_test.drop('PoolQC', axis=1)

__2) 'MiscFeature'__

This is a categorical variable. Here we have more than 96% of missing values. Basically, we can drop this column. However, there are 54 values that can potentially affect on the house price. So, we can change `NA` to `0` and all the rest values to `1`. This will convert this column into a numerical variable. 

In [15]:
X_full.MiscFeature.value_counts(dropna=False)

NaN     1406
Shed      49
Gar2       2
Othr       2
TenC       1
Name: MiscFeature, dtype: int64

In [16]:
X_full.MiscFeature.isna().sum() / X_full.shape[0]

0.963013698630137

In [17]:
X_full.MiscFeature = X_full.MiscFeature.fillna(0)
X_full.loc[X_full.MiscFeature != 0, 'MiscFeature'] = 1
cols_num.append('MiscFeature')

In [18]:
X_full.MiscFeature.value_counts(dropna=False)

0    1406
1      54
Name: MiscFeature, dtype: int64

In [19]:
# do the same with X_test
X_test.MiscFeature = X_test.MiscFeature.fillna(0)
X_test.loc[X_test.MiscFeature != 0, 'MiscFeature'] = 1

In [20]:
X_test.MiscFeature.value_counts()

0    1408
1      51
Name: MiscFeature, dtype: int64

__3) 'Alley'__

This is a categorical variable. Here we have more than 93% of missing values. 
Basically, I will drop this column.

In [21]:
X_full.Alley.isna().sum() / X_full.shape[0]

0.9376712328767123

In [22]:
X_full = X_full.drop('Alley', axis=1)
X_test = X_test.drop('Alley', axis=1)

__4) 'Fence'__

This is a categorical variable. Here we have more than 80% of missing values. 
`Nan` means that there's no fence around the house. Missing values to be imputed with `SimpleImputer` and then all values will be ordinally encoded with `OrdinalEncoder`.

In [23]:
X_full.Fence.value_counts(dropna=False)

NaN      1179
MnPrv     157
GdPrv      59
GdWo       54
MnWw       11
Name: Fence, dtype: int64

In [24]:
X_full.Fence.isna().sum() / X_full.shape[0]

0.8075342465753425

In [25]:
cols_cat_ord.append('Fence')

__5) 'FireplaceQu'__

This is a categorical variable that can be Ordinaly encoded after assigning `No` to missing values.

In [26]:
X_full.FireplaceQu.value_counts(dropna=False)

NaN    690
Gd     380
TA     313
Fa      33
Ex      24
Po      20
Name: FireplaceQu, dtype: int64

In [27]:
cols_cat_ord.append('FireplaceQu')

__6) 'LotFrontage'__

This is a numerical variable. Although there are quite many missing values, we may impute them with 'mean' value using `SimpleImputer` later.

In [28]:
X_full.LotFrontage.value_counts(dropna=False)

NaN      259
60.0     143
70.0      70
80.0      69
50.0      57
        ... 
137.0      1
38.0       1
33.0       1
150.0      1
46.0       1
Name: LotFrontage, Length: 111, dtype: int64

In [29]:
X_full.LotFrontage.describe()

count    1201.000000
mean       70.049958
std        24.284752
min        21.000000
25%        59.000000
50%        69.000000
75%        80.000000
max       313.000000
Name: LotFrontage, dtype: float64

In [30]:
cols_num.append('LotFrontage')

__7-11) GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond__

All these columns have the same amount of missing values (81). Those `NA`'s mean that there's no garage in the house. Besides that, there are differencies in types of the variables among them.

__GarageType__ is a categorical variable and I will apply One-Hot encoding to it 

In [31]:
X_full['GarageType'].value_counts(dropna=False)

Attchd     870
Detchd     387
BuiltIn     88
NaN         81
Basment     19
CarPort      9
2Types       6
Name: GarageType, dtype: int64

In [32]:
cols_cat_oh.append('GarageType')

__GarageYrBlt__ is a column with numerical values, where missing years can be imputed with mean value

In [33]:
X_full.GarageYrBlt.value_counts(dropna=False)

NaN       81
2005.0    65
2006.0    59
2004.0    53
2003.0    50
          ..
1927.0     1
1900.0     1
1906.0     1
1908.0     1
1933.0     1
Name: GarageYrBlt, Length: 98, dtype: int64

In [34]:
cols_num.append('GarageYrBlt')

__GarageFinish__ is a column with categorical values, where missing values can be imputed using `SimpleImputer`. The values can be ordinally encoded.

In [35]:
X_full.GarageFinish.value_counts(dropna=False)

Unf    605
RFn    422
Fin    352
NaN     81
Name: GarageFinish, dtype: int64

In [36]:
cols_cat_ord.append('GarageFinish')

__GarageQual__ is a column with categorical values, where missing values can be imputed using `SimpleImputer`. The values then can be ordinally encoded.

In [37]:
X_full.GarageQual.value_counts(dropna=False)

TA     1311
NaN      81
Fa       48
Gd       14
Ex        3
Po        3
Name: GarageQual, dtype: int64

In [38]:
cols_cat_ord.append('GarageQual')

__GarageCond__ is a column with categorical values, where missing values can be imputed using `SimpleImputer`. The values then can be ordinally encoded.

In [39]:
X_full.GarageCond.value_counts(dropna=False)

TA     1326
NaN      81
Fa       35
Gd        9
Po        7
Ex        2
Name: GarageCond, dtype: int64

In [40]:
cols_cat_ord.append('GarageCond')

__12-16) BsmtExposure, BsmtFinType2, BsmtFinType1, BsmtCond, BsmtQual__

Columns with information about basement. Obviously no values means no basement

__BsmtExposure__ is a column with categorical values, where missing values can be imputed using `SimpleImputer`. The values then can be ordinally encoded.

In [41]:
X_full.BsmtExposure.value_counts(dropna=False)

No     953
Av     221
Gd     134
Mn     114
NaN     38
Name: BsmtExposure, dtype: int64

In [42]:
cols_cat_ord.append('BsmtExposure')

__BsmtFinType2__ is a column with categorical values, where missing values can be imputed using `SimpleImputer`. The values then can be ordinally encoded.

In [43]:
X_full.BsmtFinType2.value_counts(dropna=False)

Unf    1256
Rec      54
LwQ      46
NaN      38
BLQ      33
ALQ      19
GLQ      14
Name: BsmtFinType2, dtype: int64

In [44]:
cols_cat_ord.append('BsmtFinType2')

__BsmtFinType1__ is a column with categorical values, where missing values can be imputed using `SimpleImputer`. The values then can be ordinally encoded.

In [45]:
X_full.BsmtFinType1.value_counts(dropna=False)

Unf    430
GLQ    418
ALQ    220
BLQ    148
Rec    133
LwQ     74
NaN     37
Name: BsmtFinType1, dtype: int64

In [46]:
cols_cat_ord.append('BsmtFinType1')

__BsmtCond__ is a column with categorical values, where missing values can be imputed using `SimpleImputer`. The values then can be ordinally encoded.

In [47]:
X_full.BsmtCond.value_counts(dropna=False)

TA     1311
Gd       65
Fa       45
NaN      37
Po        2
Name: BsmtCond, dtype: int64

In [48]:
cols_cat_ord.append('BsmtCond')

__BsmtQual__ is a column with categorical values, where missing values can be imputed using `SimpleImputer`. The values then can be ordinally encoded.

In [49]:
X_full.BsmtQual.value_counts(dropna=False)

TA     649
Gd     618
Ex     121
NaN     37
Fa      35
Name: BsmtQual, dtype: int64

In [50]:
cols_cat_ord.append('BsmtQual')

__17-18) MasVnrArea, MasVnrType__

Only 8 missing values and these 8 missing values are in the same rows

In [51]:
X_full.MasVnrArea.value_counts(dropna=False)

0.0      861
NaN        8
72.0       8
108.0      8
180.0      8
        ... 
562.0      1
89.0       1
921.0      1
762.0      1
119.0      1
Name: MasVnrArea, Length: 328, dtype: int64

In [52]:
X_full.MasVnrType.value_counts(dropna=False)

None       864
BrkFace    445
Stone      128
BrkCmn      15
NaN          8
Name: MasVnrType, dtype: int64

In [53]:
X_full[X_full.MasVnrType.isna() | X_full.MasVnrArea.isna()][['MasVnrType','MasVnrArea']]

,MasVnrType,MasVnrArea
Id,,
235,NaN,NaN
530,NaN,NaN
651,NaN,NaN
937,NaN,NaN
974,NaN,NaN
978,NaN,NaN
1244,NaN,NaN
1279,NaN,NaN


So, missing values in numerical 'MasVnrArea' will be replaced with '0.0' and in categorical 'MasVnrType' with 'None' in both train and test datasets

In [54]:
X_full.MasVnrType = X_full.MasVnrType.fillna('None')
X_test.MasVnrType = X_test.MasVnrType.fillna('None')

cols_cat_oh.append('MasVnrType')

X_full.MasVnrArea = X_full.MasVnrArea.fillna(0.0)
X_test.MasVnrArea = X_test.MasVnrArea.fillna(0.0)

cols_num.append('MasVnrArea')

__19) Electrical__

Categorical variable with only 1 missing value which can be imputed with most frequent value (SBrkr). Then good to be one-hot encoded.

In [55]:
X_full.Electrical.value_counts(dropna=False)

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
NaN         1
Name: Electrical, dtype: int64

In [56]:
X_full.Electrical = X_full.Electrical.fillna(X_full.Electrical.mode().iloc[0])
X_test.Electrical = X_test.Electrical.fillna(X_test.Electrical.mode().iloc[0])

#### Also found residuals in some variables that spoil the transform, so need to remove them

In [57]:
X_test.loc[X_test.HeatingQC == 'Po', 'HeatingQC'] = 'Fa'
X_full.loc[X_full.HeatingQC == 'Po', 'HeatingQC'] = 'Fa'

X_test.loc[X_test.ExterCond == 'Po', 'ExterCond'] = 'Fa'
X_full.loc[X_full.ExterCond == 'Po', 'ExterCond'] = 'Fa'

X_test.loc[X_test.BsmtCond == 'Po', 'BsmtCond'] = 'Fa'
X_full.loc[X_full.BsmtCond == 'Po', 'BsmtCond'] = 'Fa'

X_test.loc[X_test.GarageCond == 'Ex', 'GarageCond'] = 'Gd'
X_full.loc[X_full.GarageCond == 'Ex', 'GarageCond'] = 'Gd'

X_full.loc[X_full.HouseStyle == '2.5Fin', 'HouseStyle'] = '2Story'

#### Now all the columns with missing values either deleted or updated and/or splitted into different groups of variables. 

Note that for numerical values we can impute all left missing values with 'mean' value, so we don't need our `cols_num` list anymore as we apply the same `SimpleImputer` to all the numerical variables.

For categorical variables we will impute missing values with 'missing_values' also using `SimpleImputer`.

But, we discovered only the columns with missing values, so need to proceed with all the rest.

#### A list of all numerical variables:

In [58]:
num_cols = [col for col in X_full.columns if X_full[col].dtype in ['int64', 'float64']]

#### Categorical variable should be divided by the type on encoding:

In [59]:
cols_to_be_divided = [col for col in X_full.columns if X_full[col].dtype == 'object'
                      and col not in (cols_cat_ord + cols_cat_oh)]

In [60]:
for col in cols_to_be_divided:
    if 'Gd' in X_full[col].values or 'GLQ' in X_full[col].values:
        cols_cat_ord.append(col)
    else:
        cols_cat_oh.append(col)    

#### So, now we have 3 groups: numerical variables, categorical for ordinal encoding and categorical for one-hot encoding

In [61]:
# num_cols
# cols_cat_ord
# cols_cat_oh

In [62]:
len(num_cols + cols_cat_oh + cols_cat_ord)

77

#### We keep in mind that there's a column 'Neighborhood' with 25 unique values and it is high number for One-Hot Encoding, but we'll see

In [63]:
object_nunique = list(map(lambda col: X_full[col].nunique(), cols_cat_oh))
d = dict(zip(cols_cat_oh, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('MiscFeature', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('MasVnrType', 4),
 ('LotShape', 4),
 ('LandContour', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('Electrical', 5),
 ('GarageType', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('SaleCondition', 6),
 ('HouseStyle', 7),
 ('Functional', 7),
 ('Condition2', 8),
 ('RoofMatl', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

## Now, I put preprocessing and model together into the pipeline

#### Step 1: Define Preprocessing Steps

In [64]:
# preprocessing for numerical columns
numerical_transformer = SimpleImputer(strategy='mean')

# preprocessing for categorical columns
categorical_ordinal_transformer = Pipeline(steps = [('imputer', SimpleImputer(strategy='most_frequent')),
                                                     ('ordinal', OrdinalEncoder())])

categorical_one_hot_transformer = Pipeline(steps = [('imputer', SimpleImputer(strategy='most_frequent')),
                                                     ('one_hot', OneHotEncoder(handle_unknown='ignore'))])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, num_cols),
                                               ('cat_ord', categorical_ordinal_transformer, cols_cat_ord),
                                               ('cat_oh', categorical_one_hot_transformer, cols_cat_oh)])                           


#### Step 2: Define the Model

In [65]:
# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_full, y)

In [66]:
# define XGBRegressor model
model = XGBRegressor(n_estimators=400, learning_rate=0.05, n_jobs=4, random_state=0)

#### Step 3: Create and Evaluate a Pipeline

In [67]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)])

# Preprocessing of training data, fit model
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

#### Step 4: Check accuracy

Our metric is Root-Mean-Squared-Error (RMSE), so we can use `mean_squared_error` from `sklearn.metrics` using parameter `squared=False`

In [68]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_valid, preds, squared=False)

print(f'RMS: {rms}')

RMS: 24177.978638028664


In [69]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_valid, preds)
print('MAE:', mae)

MAE: 16524.73957619863


In [70]:
# for n in range(100, 800, 100):
#     model = XGBRegressor(n_estimators=n, learning_rate=0.05, n_jobs=4, random_state=0)
#     # Bundle preprocessing and modeling code in a pipeline
#     my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                                   ('model', model)])

#     # Preprocessing of training data, fit model
#     my_pipeline.fit(X_train, y_train)

#     # Preprocessing of validation data, get predictions
#     preds = my_pipeline.predict(X_valid)
#     rms = mean_squared_error(y_valid, preds, squared=False)
#     mae = mean_absolute_error(y_valid, preds)
    
#     print(f'n_estimators: {n}: MAE: {mae}, RMS: {rms}')

In [71]:
my_pipeline.fit(X_full, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBath',
                                                   'HalfBath', 'Bed...
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.05, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=400, n_jobs=4, num_parallel_tree=1,
                              predictor='auto', random_state=0, reg_alpha=0,
                              reg_lambda=1, ...))])

In [72]:
test_predictions = my_pipeline.predict(X_test)

In [73]:
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': test_predictions})
output.to_csv('data/submission_1.csv', index=False)